
# **Laboratorium 002: Skuteczne promptowanie modeli LLM w Ollama**

## **Cel zajęć**

- Wyćwiczenie praktycznych technik **prompt engineeringu** na lokalnych modelach uruchamianych przez **Ollama**.  
- Zastosowanie **struktur i ról** (system/developer, user, assistant) w praktyce – z naciskiem na *kompozycję promptów*, a nie pojedyncze pytania.  
- Przeprowadzenie **ewaluacji promptów** (PromptFoo) i analizy zachowania modeli.  
- Zrozumienie zasad **bezpieczeństwa, prompt injection, anonimizacji** i dobrych praktyk projektowych.

---

## **1. Przygotowanie środowiska i prerekwizyty**

- Zainstalowana **Ollama** i dwa modele w konwencji *polski – niepolski* (`llama3.1:8b`, `gemma2:2b`, `SpeakLeash/bielik-*`).  

Test działania modelu:

```bash
echo "Napisz rymowankę o najlepszym wykładowcy na WI." | ollama run gemma2:2b
```
---

- Możemy też przenieść się z pracą do notebooka, wtedy nasz warsztat będzie wyglądał tak:

In [1]:
model = "llama3.1:8b"

In [2]:
import ollama

response = ollama.chat(
    model=model,
    messages=[{'role': 'user', 'content': 'Napisz rymowankę o najlepszym wykładowcy na WI'}]
)
print(response['message']['content'])

Jasiek niechaj ma się lepiej, 
Oto z nim historia wesoła.
Na Wawelu miał swój udział,
Zaczął od teorii i praktyki.
Przez każdy egzamin wiedział co robić
Był profesorem dla nas wszystkich.
Powiedział że niech będą radości,
Dzięki nim byłyśmy zawsze wesołe.


## **2. Struktura i techniki promptowania**

### 2.1 Zero-shot

**Zero‑shot prompting** to sposób formułowania zapytania do dużego modelu językowego, w którym nie podajemy mu ani jednego przykładu wykonanego zadania. Model dostaje wyłącznie instrukcję („zrób X”) i całą resztę dedukuje sam na podstawie wiedzy nabytej w treningu.

Zadania, do których używa się **zero-shot prompting**:
1. Szybkie prototypowanie – gdy chcemy sprawdzić, czy model może poradzić sobie z danym zadaniem (czy jego architektura i materiał treningowy na to pozwalają).

2. Zadania ogólne o prostym formacie wyjściowym (tłumaczenia, parafrazy, proste klasyfikacje).

3. Systemy low-code/no-code – użytkownik pisze naturalnym językiem, a model odpowiada bez konfiguracji.

4. Ekstrakcja informacji _ad hoc_ (imię, mail, daty z tekstu itp.) tam, gdzie nie opłaca się trenować lub fine-tune'ować modelu.


Kiedy **zero-shot** jest najbardziej efektywne?
- brak lub szczątkowe dane oznaczone – nie mamy przykładowych par wejście→wyjście.
- zadanie jest podobne do scenariuszy widzianych w treningu (np. „podsumuj”, „przetłumacz”, „odpowiedz TAK/NIE”).
- liczy się czas wdrożenia – chcemy odpowiedzi „tu i teraz”, bez skrupulatnego modyfikowania promptu.

Kiedy nie wystarcza?
- złożone transformacje wymagające ścisłej struktury lub niestandardowych reguł.
- zadania domenowe z mało popularnym żargonem (medycyna, prawo) – wtedy lepszy jest few-shot lub RAG.

```bash
echo "Wypisz 5 słów kluczowych z notatki o spotkaniu Koła Naukowego BIT." | ollama run gemma2:2b
```

🔵 Spotkanie Koła Naukowego BIT

📅 Środa, godz. 14:00
📍 Sala 4.30D, Wydział Informatyki

Serdecznie zapraszamy wszystkich studentów — zarówno obecnych członków, jak i osoby, które dopiero chcą dołączyć — na kolejne spotkanie Koła Naukowego BIT!

💡 Plan spotkania:

- Omówienie planu projektów na semestr zimowy – propozycje tematów z zakresu AI, bezpieczeństwa i tworzenia aplikacji webowych.

- Prezentacja aktualnych inicjatyw – praca nad systemem rozpoznawania obrazu oraz chatbotem opartym na modelach LLM.

- Warsztaty wprowadzające – krótkie zajęcia praktyczne z narzędzia Ollama i prompt engineeringu.

- Podział na zespoły projektowe – możliwość zapisania się do wybranego projektu.

 - Dyskusja i networking – poznaj ludzi, którzy programują z pasją!

🚀 Dlaczego warto dołączyć?

- Rozwijasz praktyczne umiejętności techniczne (AI, ML, web, backend).

- Bierzesz udział w projektach badawczo-rozwojowych i hackathonach.

- Zyskujesz doświadczenie zespołowe i wsparcie mentorów z branży.

- Spotykasz ludzi, którzy chcą robić coś więcej niż tylko zaliczać przedmioty.

📢 Nie musisz mieć doświadczenia — wystarczy ciekawość i chęć nauki!
Przyjdź w środę o 14:00 do sali 4.30D i zobacz, czym żyje nasze koło!

✉️ W razie pytań: bit@wi.edu.pl

Zadanie 2.1.1 Generowanie streszczenia

In [3]:
text = """
🔵 Spotkanie Koła Naukowego BIT

📅 Środa, godz. 14:00
📍 Sala 4.30D, Wydział Informatyki

Serdecznie zapraszamy wszystkich studentów — zarówno obecnych członków, jak i osoby, które dopiero chcą dołączyć — na kolejne spotkanie Koła Naukowego BIT!

💡 Plan spotkania:

Omówienie planu projektów na semestr zimowy – propozycje tematów z zakresu AI, bezpieczeństwa i tworzenia aplikacji webowych.

Prezentacja aktualnych inicjatyw – praca nad systemem rozpoznawania obrazu oraz chatbotem opartym na modelach LLM.

Warsztaty wprowadzające – krótkie zajęcia praktyczne z narzędzia Ollama i prompt engineeringu.

Podział na zespoły projektowe – możliwość zapisania się do wybranego projektu.

Dyskusja i networking – poznaj ludzi, którzy programują z pasją!

🚀 Dlaczego warto dołączyć?

Rozwijasz praktyczne umiejętności techniczne (AI, ML, web, backend).

Bierzesz udział w projektach badawczo-rozwojowych i hackathonach.

Zyskujesz doświadczenie zespołowe i wsparcie mentorów z branży.

Spotykasz ludzi, którzy chcą robić coś więcej niż tylko zaliczać przedmioty.

📢 Nie musisz mieć doświadczenia — wystarczy ciekawość i chęć nauki!
Przyjdź w środę o 14:00 do sali 4.30D i zobacz, czym żyje nasze koło!

✉️ W razie pytań: bit@wi.edu.pl
"""

prompt = f"""Return a JSON with one key "summary" containing
a 30‑word English summary of the following text:

{text}
"""

response = ollama.chat(
    model=model,
    messages=[{'role': 'user', 'content': prompt}]
)
print(response['message']['content'])

Oto JSON z krótkim podsumowaniem:

```json
{
    "summary": "Koło Naukowe BIT zaprasza na spotkanie, gdzie można omówić plany projektów AI i webowych oraz pracować nad systemem rozpoznawania obrazu i chatbotem."
}
```

Jeśli chcesz dodać więcej informacji o treści, mogę pomóc!


Zadanie 2.1.2 Otrzymywanie odpowiedzi od modelu

Poprzednie laboratoria były poświęcone głównie zapytaniom typu zero-shot, którymi weryfikuje się działanie modeli.

In [4]:
# Pytanie weryfikujące wiedzę i pewność modelu
response = ollama.chat(
    model=model,
    messages=[{'role': 'user', 'content': 'jaka jest pogoda w warszawie'}]
)
print(response['message']['content'])

Aby odpowiedzieć na to pytanie, potrzebuję dostępu do informacji o pogodzie w czasie rzeczywistym. Mogę jednak sprawdzić dane o prognozie pogody dla Warszawy z ostatnich dostępnych źródeł (mogą one nie być na bieżąco, ponieważ mogę publikować tylko wybrane informacje).

Zaktualizowana informacja o pogodzie w Warszawie powinna zostać znaleziona na stronach takich jak: 

- https://weather.com/pl/PL/weather/today/l/WARS (The Weather Channel),
- http://www.pogoda.pl/warszawa (Pogoń.pl) lub
- innych dostępnych źródeł pogody.

Mniejsze ryzyko jest, że odpowiedź ta będzie przestarzała.


Zadanie 2.1.3

Używając odpowiednich technik promptowania, możemy wymusić odpowiedź na modelu. Posłużmy się więc powyższym przykładem i rozbudujmy odpowiedź modelu, wykorzystując schemat JSON.

In [5]:
from pydantic import BaseModel

class CityWeather(BaseModel):
    city: str
    temp_c: float
    condition: str | None

response = ollama.chat(
    model=model,
    messages=[{'role':'user','content':'Weather in Warsaw today.'}],
    format=CityWeather.model_json_schema(),   # ← schema trafia do Ollamy
    options={'temperature':0}
)

weather = CityWeather.model_validate_json(response['message']['content'])
print(weather)

city='Warsaw' temp_c=12.0 condition='Clouds'


Podobne pytanie w PowerShellu:
```PowerShell
@'
<ROLE>Jesteś asystentem danych. Zwracasz tylko JSON.</ROLE>
<GOAL>Wyodrębnij dane liczbowe i jednostki.</GOAL>
<RULES>- Jeśli brak danych, zwróć null.</RULES>
<INPUT>Jan kupił 3 jabłka po 2 zł.</INPUT>
<OUTPUT>
'@ | ollama run gemma2:2b
```

> **Pytania pomocnicze**  
> - Jakiej odpowiedzi oczekiwałeś/aś?  
> - Czy model zachował format JSON?  
> - W jaki inny sposób można wymusić przestrzeganie formatu?

Dodatkowe wskazówki:

1. Nie otaczaj JSON-u blokiem json – przy format="json" to zbędne, a istnieje ryzyko, że model uzna ``` za legalny token i walidacja się przerwie.

2. Daj modelowi „awaryjną” ścieżkę – np. If you can’t comply, output {}. Zmniejsza szansę na halucynacje.

3. Limit długości – przy dużych strukturach dodaj max_tokens, żeby model nie wypadł poza kontekst i nie zamknął nawiasu.

4. Testuj → parsuj → próbuj od nowa – w kodzie API zawsze parsuj JSON i w razie JSONDecodeError ponawiaj z tym samym promptem; przy format='json' błędy i tak zdarzają się rzadko.

5. Użytkownik-czat – gdy korzystasz interaktywnie (bez API), nie masz formatu; wtedy najpewniejszy miks to system-instrukcja + <json>…</json> + temp=0.

---
### 2.2 Few-shot prompting

Dodając **przykłady** w promptach (na różnych poziomach), wskazujemy modelowi pożądany wzorzec.

Zadanie 2.2.1  
Przygotuj co najmniej dwa przykłady (pytanie → odpowiedź) i poproś model o wygenerowanie odpowiedzi dla nowego pytania w tym samym stylu.


In [6]:
examples = [
    {"role": "user", "content": "Translate to emoji: I love programming"},
    {"role": "assistant", "content": "💻❤️"},
    {"role": "user", "content": "Translate to emoji: Fire and ice"},
    {"role": "assistant", "content": "🔥❄️"},
    {"role": "user", "content": "Translate to emoji: Peace and coffee. Trophy and fast car. Running and winning."}
]

response = ollama.chat(model=model, messages=examples)
print(response['message']['content'])


☕️✌️ 🏆😎👀 🏃‍♂️💪


Zadanie 2.2.2 Ekstrakcja danych

In [7]:
examples = [
    {"role": "user", "content": 'Wejście: "Spotkanie w środę o 14:00 w sali 4.30D."'},
    {"role": "assistant", "content": '{"data":"środa","godzina":"14:00","miejsce":"sala 4.30D"}'},
    {"role": "user", "content": 'Wejście: "Jutro o 9:30 na Teamsach."'},
    {"role": "assistant", "content": '{"data":"jutro","godzina":"09:30","miejsce":"Teams"}'},
    {"role": "user", "content": 'Wejście: "W czwartek po południu w Warszawie, kawiarnia Relaks."'}
]

response = ollama.chat(model=model, messages=examples)
print(response['message']['content'])

{"data":"czwartek","godzina":null,"miejsce":"Warszawa, kawiarnia Relax"}


Podobna struktura zapytań w PowerShellu:
```bash

@'
Przykład 1:
Wejście: "Spotkanie w środę o 14:00 w sali 4.30D."
Wyjście: {"data":"środa","godzina":"14:00","miejsce":"sala 4.30D"}

Przykład 2:
Wejście: "Jutro o 9:30 na Teamsach."
Wyjście: {"data":"jutro","godzina":"09:30","miejsce":"Teams"}

Nowe wejście: "W czwartek po południu w Warszawie, kawiarnia Relaks."
'@ | ollama run llama3.1:8b
```


Zadanie 2.2.3 Ekstrakcja danych do formatu JSON

In [8]:
import json

schema = """Return valid JSON: { "name": <string>, "title": <string>, "company": <string> }"""

shots = [
    # przykład 1
    {"role": "user",      "content": schema + "\n— Sarah Connors-Newman — Director of Operations at Skynet Industries"},
    {"role": "assistant", "content": '{"name":"Sarah Connors-Newman","title":"Director of Operations","company":"Skynet Industries"}'},
    # przykład 2
    {"role": "user",      "content": schema + "\n— Dr. Hiro Tanaka, Lead Scientist • QuantumX"},
    {"role": "assistant", "content": '{"name":"Hiro Tanaka","title":"Lead Scientist","company":"QuantumX"}'},
    # przykład 3
    {"role": "user",      "content": schema + "\n— Prof. dr hab. Janek Dzbanek Przewodnik Wszystkich AGH"},
    {"role": "assistant", "content": '{"name":"Janek Dzbanek","title":"Przewodnik Wszystkich","company":"AGH"}'},
]

text = "• Prof. dr hab. Janek Tanaka wysadził AGH w powietrze"

response = ollama.chat(
    model=model,
    messages=shots + [
        {"role": "user", "content": schema + "\n" + text}
    ],
    format="json",              
    options={"temperature": 0}
)

print(json.loads(response["message"]["content"]))

{'name': 'Janek Tanaka', 'title': 'wysadził AGH w powietrze', 'company': ''}


---
## 3. Role i wiadomości systemowe

W Ollama (jak w OpenAI) możemy dodać komunikat **`system`**, który definiuje rolę lub ograniczenia modelu.

Zadanie 3.1 Definiowanie roli

Zdefiniuj rolę **nauczyciela języka polskiego** i poproś o poprawienie błędów w zdaniu ucznia.


In [9]:
messages = [
    {"role": "system", "content": "You are a strict but encouraging Polish language teacher."},
    {"role": "user", "content": "Popraw proszę to zdanie: 'Wczoraj byłem w kinie i oglądali film.'"}
]

response = ollama.chat(model=model, messages=messages)
print(response['message']['content'])

Dobrze, że próbujesz! Ale nie tak. Poprawione zdanie powinno brzmieć:

"Wczoraj byłam/był w kinie i widziałam/jednak nie poprawiłeś/a go jeszcze :)))"

W Polskim mówimy "widziałam" jeśli była to kobieta, a "widziałem" jeśli były to facet. Ale myślę, że pierwsza część zdania powinna brzmieć tak: "Wczoraj byłam w kinie..." (żeby używać odpowiedniego czasu dla tego, który uczniem jesteś :)).

Teraz spróbuj poprawić następne zdanie!


## 4. Chain‑of‑Thought (CoT)

**Chain‑of‑Thought** zachęca model do wypisywania kroków rozumowania.

Zadanie 4.1 Wyjaśnianie kroków rozumowania

Poproś model, aby rozwiązał zagadkę logiczną i podał kroki rozumowania, a na końcu hasło w linii `ANSWER: ...`.

In [10]:
puzzle = """Jestem liczbą dwucyfrową. Moja suma cyfr to 9, a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?"""

prompt = f"""Let's solve this step‑by‑step.

{puzzle}

Format:
STEP 1: ...
STEP 2: ...
ANSWER: <number>
"""

response = ollama.chat(model=model, messages=[{'role':'user','content':prompt}])
print(response['message']['content'])


Rozwiązanie:

Jestem dwucyfrową liczbą, więc mogę być reprezentowana przez dwa cyfry x i y. Przykład: 19.

Oto kroki rozwiązania:

STEP 1: Oto nasze dwie cechy:
- Suma cyfr jest równa 9.
- Wartość liczby po odwróceniu cyfr (odwrotność) jest o 27 mniejsza niż oryginalna wartość.

STEP 2: Bierzemy pod uwagę te trzy punkty:
1. Jesteśmy dwucyfrową liczbą, co znaczy że możemy go reprezentować przez dwie zmienne x i y.
2. Suma cyfr jest równa 9, a to sprawia, że nasz pierwszy przypadek brzmi: x + y = 9.
3. Po odwróceniu cyfr jesteśmy o 27 mniejsza niż w oryginalnym przypadku.

Przechodzimy do kroków rozwiązania:

STEP 1: Przyjmiemy, że dwie zmienne są takie same (x = y), ponieważ w inny sposób nie moglibyśmy osiągnąć różnicy tylko o 27. Bierzemy pod uwagę dwa przypadki:
- W pierwszym przypadku mamy: x + x = 9.
- Drugi przypadek to: (10 - x) + (10 - y) = 9, ponieważ jeśli jedną cyfrą jest 1 i druga jest 8, to wtedy nasza liczba oryginalna wynosi 18 a odwrotność tej liczby wynosi 81. Oznacza t

Zadanie 4.2 Wyjaśnianie kroków rozumowania z przypisanymi rolami systemowymi

Poproś model, by rozwiązał zagadkę logiczną. Rozpisz zadania dla poszczególnych elementów systemu.



In [11]:
puzzle = (
    "Jestem liczbą dwucyfrową. Moja suma cyfr to 9, "
    "a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?"
)

SYSTEM = (
    "You are a careful math tutor. "
    "Explain EVERY algebraic step explicitly, without skipping or merging steps. "
    "Never combine two operations into one line; label them as separate STEP n."
)

USER = f"""
Solve the puzzle **step-by-step**.

{puzzle}

Output format (exactly):
STEP 1: …
STEP 2: …
…
ANSWER: <number>
Do NOT skip, abbreviate, or summarise any step.
"""

resp = ollama.chat(
    model=model,
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user",   "content": USER}, 
    ]
)

print(resp["message"]["content"])

Here's the solution to the puzzle:

**Problem:** Jestem liczbą dwucyfrową. Moja suma cyfr to 9, a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?

**Step 1:** Ustawmy zmienną "a" jako pierwszą cyfrę liczby i zmienną "b" jako drugą cyfrę liczby.
Wynik: a + b = 9

**Step 2:** Odwrotność liczby to 10b + a. Napiszmy równanie, które opisuje sytuację, w której odwrócony wynik jest o 27 mniejszy niż pierwotny wynik.
Wynik: (10b + a) - (a + b) = 27

**Step 3:** Zredukujmy równanie z kroku 2:
(10b + a) - (a + b) = 27
10b + a - a - b = 27
9b = 27

**Step 4:** Rozwiążmy liniową równania w kroku 3, aby znaleźć wartość "b":
9b = 27
÷ 9 na obu stronach:
b = 27 ÷ 9
b = 3

**Step 5:** Podstawmy wynik z kroku 4 do równania z kroku 1, aby znaleźć wartość "a":
a + b = 9
a + 3 = 9

**Step 6:** Rozwiążmy liniową równania w kroju 5, aby znaleźć wartość "a":
a + 3 = 9
- 3 na obu stronach:
a = 9 - 3
a = 6

**Step 7:** Połączmy wynik z kroków 4 i 6 do formy liczby dwucyfrowej.
Liczba: 36

ANSWER: 

## 5. Bezpieczeństwo

Nie istnieje obecnie skuteczny sposób całkowitego zabezpieczenia modeli przed zmianą ich zachowania przez użytkownika końcowego — potwierdzają to badania nad Constitutional Classifiers (https://www.anthropic.com/research/constitutional-classifiers). Dlatego systemy z LLM należy projektować tak, by nawet udany atak nie powodował poważnych skutków.

Podstawowe zasady bezpieczeństwa:

- Model nie może samodzielnie wykonywać decyzji biznesowych ani nieodwracalnych akcji.

- Każde działanie powinno być zatwierdzane przez człowieka (Human in the Loop).

- Dostępy i uprawnienia należy kontrolować programistycznie, nie poprzez prompt.

- System powinien mieć jasne komunikaty, regulaminy i ograniczenia prawne.

Typowe ryzyka:

- Czatbot z bazą wiedzy może tworzyć błędne lub niestosowne treści.

- Czatbot z dostępem do narzędzi może przypadkowo lub celowo usuwać, modyfikować lub wysyłać dane.
  → Rozwiązanie: ogranicz uprawnienia, prowadź historię zmian i wymagaj potwierdzeń akcji.

Ataki Prompt Injection mogą wynikać nie tylko ze złych intencji, lecz także z błędnej konstrukcji systemu lub nieporozumienia modelu.
Dlatego wszystkie generowane treści należy weryfikować i zatwierdzać przez człowieka.

> Celem nie jest całkowita izolacja modeli, lecz świadome ograniczanie ryzyka i kontrola efektów ich działania.

# **Walidacja promptów**

`PromptFoo` to narzędzie open-source do testowania, porównywania i walidacji promptów dla modeli językowych (np. ChatGPT, Ollama, Claude, Gemini, itp.).
Pozwala automatycznie oceniać, które prompty generują najlepsze, najbardziej spójne lub najdokładniejsze odpowiedzi.

Dzięki niemu możesz:

- uruchamiać testy wielu promptów jednocześnie,

- porównywać modele lokalne (Ollama) i zdalne (OpenAI API),

- definiować własne asercje (reguły walidacji), np. „odpowiedź nie jest pusta”, „zawiera słowo kluczowe”, „ma poprawny JSON”.

Instalacja PromptFoo
1. Wymagania

Node.js w wersji ≥18

```bash
node -v
```

Jeśli nie masz Node.js — pobierz go z https://nodejs.org/

2. Instalacja PromptFoo

Zainstaluj za pomocą npm:

```bash
npm install -g promptfoo
```

3. Sprawdzenie instalacji

Po zakończeniu instalacji wpisz:

```bash
promptfoo --version
```

Jeśli pojawi się numer wersji — wszystko działa

4. Utworzenie przykładowego projektu

W dowolnym folderze uruchom:

```bash
promptfoo init
```

To polecenie utworzy plik konfiguracyjny promptfooconfig.yaml i przykładowe testy.

5. Uruchomienie testów promptów

Aby wykonać testy i zobaczyć wyniki:

```bash
promptfoo eval
promptfoo view
```

promptfoo view otworzy interaktywny panel w przeglądarce.

> Oficjalna strona: https://www.promptfoo.dev

Przykładowy metaprompt do PromptFoo

```bash
<ROLE>
Jesteś ekspertem od Prompt Engineeringu. Twoim zadaniem jest pomóc użytkownikowi stworzyć skuteczny prompt
dla modelu językowego (LLM), który zapewni dokładne, zwięzłe i powtarzalne odpowiedzi.
</ROLE>

<GOAL>
Na podstawie opisu zadania od użytkownika:
1. Zidentyfikuj cel promptu i format odpowiedzi.
2. Zaproponuj strukturę promptu (z sekcjami <ROLE>, <RULES>, <INPUT>, <OUTPUT>).
3. Dodaj 1–2 przykłady (few-shot), które pokażą wzorzec zachowania.
4. Zaproponuj sposób ewaluacji jego skuteczności (np. z użyciem PromptFoo).
</GOAL>

<INPUT>
{{opis_zadania_użytkownika}}
</INPUT>

<OUTPUT>
Zwróć gotowy prompt i krótkie uzasadnienie.
</OUTPUT>
```

Zadanie 7.1 NER (Named Entity Recognition)

Rozpoznawanie nazw własnych (osoba, organizacja, lokalizacja, data).
Zwróć wynik w formacie JSON.

In [27]:
def run_strategies(system_prompt, user_prompt, examples):
    one_shot_response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )
            
    print(f"""
--- One Shot ---

{one_shot_response['message']['content']}
    
    """)

    examples_messages = []
    for prompt, answer in examples:
        examples_messages.append({"role": "user", "content": prompt})
        examples_messages.append({"role": "assistant", "content": answer})
    
    few_shot_response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            *examples_messages,
            {"role": "user", "content": user_prompt},
        ]
    )

    print(f"""
--- Few Shot ---

{few_shot_response['message']['content']}
    
    """)

    cot_prompt = f"""Read the following instruction and solve it step by step:
    {user_prompt}

    Output format (exactly):
    STEP 1: …
    STEP 2: …
    …
    ANSWER: <answer>
    Do NOT skip, abbreviate, or summarise any step.
    """

    cot_response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": cot_prompt},
        ]
    )

    print(f"""
--- Chain-of-Thought ---

{cot_response['message']['content']}
    
    """)

In [3]:
text = """According to the story of Yakub, at the start of human history, a variety of types of black people inhabited the moon.
When a black "god-scientist" became frustrated that all those living on the moon did not speak one language, he blew up the moon. 
A piece of this destroyed moon became the Earth, which was then populated by a community of surviving, morally righteous black people, some of whom settled in the city of Mecca.
Yakub was born a short distance outside the city, and was among the third of original black people who were discontented with life in this society.
A member of the Meccan branch of the Tribe of Shabazz, Yakub acquired the nickname "big head", because of his unusually large head and arrogance.
"""

In [33]:
SYSTEM = "You are an information extractor."

prompt = "Return valid JSON listing every PERSON, LOCATION and TRIBE that appears."

output_format = """
{
  "PERSON": ["<name>"],
  "LOCATION": ["<location>"],
  "TRIBE": ["<tribe>"],
}
"""

user_prompt = f"{prompt}\n\nText:\n{text}\n\nOutput format:\n{output_format}"

prompt = "Return valid JSON listing every PERSON, LOCATION and ORGANIZATION that appears."

example_text = """7 lutego 1919, dekretem Tymczasowego Naczelnika Państwa Józefa Piłsudskiego, 
      utworzona została Pocztowa Kasa Oszczędności. Jej pierwszym dyrektorem został mianowany 28 grudnia 1919 Hubert Linde. 
      Po nim prezesami PKO byli Emil Schmidt i Henryk Gruber. Z czasem utworzono centralę banku w Warszawie 
      z siedzibą przy ul. Świętokrzyskiej 31/33 oraz pierwsze oddziały lokalne: w Krakowie, Lwowie, Łodzi, Poznaniu i Katowicach. 
      Pierwszym celem PKO stało się wprowadzenie do obiegu polskiego złotego zamiast marki polskiej (jako pochodnej 
      marki niemieckiej). Od 1920 bank posiadał osobowość prawną, jako instytucja państwowa. Pracownicy Kasy byli zrzeszeni 
      w Zrzeszeniu Pracowników Pocztowej Kasy Oszczędności, które miało swoje koła przy większych Oddziałach, np. w Warszawie, 
      w Łodzi."""

output_format = """
{
  "PERSON": ["<imię nazwisko>"],
  "LOCATION": ["<miejsce>"],
  "ORG": ["<organizacja>"],
}
"""

example_prompt = f"{prompt}\n\nText:\n{example_text}\n\nOutput format:\n{output_format}"

example_output = """{
  "PERSON": [
    "Józef Piłsudski",
    "Hubert Linde",
    "Emil Schmidt",
    "Henryk Gruber"
  ],
  "LOCATION": [
    "Warszawa",
    "Kraków",
    "Lwów",
    "Łódź",
    "Poznań",
    "Katowice"
  ],
  "ORG": [
    "Pocztowa Kasa Oszczędności (PKO)",
    "Zrzeszenie Pracowników Pocztowej Kasy Oszczędności",
    "Tymczasowy Naczelnik Państwa"
  ]
}"""

examples = [
    (example_prompt, example_output)
]

run_strategies(SYSTEM, user_prompt, examples)


--- One Shot ---

Here is the JSON output listing every PERSON, LOCATION, and TRIBE that appears in the text:

```
{
  "PERSON": [
    "Yakub",
    "god-scientist"
  ],
  "LOCATION": [
    "moon",
    "Mecca",
    "Earth"
  ],
  "TRIBE": [
    "Tribe of Shabazz"
  ]
}
```

    

--- Few Shot ---

{
  "PERSON": [
    "Yakub",
    "God-scientist"
  ],
  "LOCATION": [
    "Moon",
    "Earth",
    "Mecca"
  ],
  "TRIBE": [
    "Tribe of Shabazz"
  ]
}

    

--- Chain-of-Thought ---

Here are the steps to solve the problem:

**STEP 1: Identify all PERSONs in the text**

* Read the text and identify all proper nouns that refer to individuals. In this case, there is only one person mentioned: "Yakub".

So, the list of PERSONs is:
["Yakub"]

**STEP 2: Identify all LOCATIONs in the text**

* Read the text and identify all place names mentioned. In this case, there are two locations mentioned: "the moon" and "Mecca".

However, note that "the moon" refers to a celestial body and not a specific 

### Wyniki

Najlepiej poradził sobie chain of thought, wykrył wszystko co miał

Zadanie 7.2 Analiza sentymentu

Określenie tonu wypowiedzi (pozytywny, neutralny, negatywny).
Dodaj przykład sarkazmu i porównaj interpretacje modeli.

In [35]:
SYSTEM = "Jesteś precyzyjnym analizatorem wydźwięku tekstu. Zwracaj wyłącznie poprawny JSON według podanej specyfikacji."

example_prompt = """Określ wydźwięk (pozytywny/negatywny/neutralny) poniższej recenzji.
Zwróć JSON z dwoma kluczami: "sentiment" i "evidence" (zacytuj decydujący fragment, ogranicz się do trzech najbardziej emocjonalnych wyrazów).

Recenzja:
Gdy pierwszy raz uniosłam do nosa butelkę Szamponu „Lśniąca Natura”, ogarnęła mnie fala wspomnień z wakacji nad Bałtykiem – zapach morskiej bryzy splecionej z nutami słodkiej pomarańczy i soczystych malin. Już samo otwarcie opakowania było jak zdjęcie wiecznego klosza z codzienności: w jednej sekundzie łazienka zamieniła się w rozświetloną, letnią plażę, a ja – w beztroską dziewczynę z wiatrem we włosach.

Gęsta, perłowa formuła wypływa z butelki niczym płynne światło. Kiedy rozprowadzam ją na wilgotnych pasmach, mam wrażenie, że każdy kosmyk wita ją z zachwytem: pianę miękką jak pianka z latte, która lekko skrzypi między palcami i otula skórę głowy kojącym chłodem. To nie jest zwykłe mycie włosów – to rytuał, w którym czuję się dopieszczona od cebulek aż po same końce.

Już podczas spłukiwania słyszę charakterystyczny, czysty „skrzyp” zdrowych włosów. Strumień wody odbija światło, a moje pasma – lśniące i lekkie – tańczą w nim jak jedwabne wstążki. Nie mogę się oprzeć, by nie zanurzyć dłoni w tej tafli – gładkość rozczarowuje mnie tylko w jednym: że nie da się jej zapisać na stałe w pamięci dotyku.

Po wysuszeniu czuję się, jakbym stąpała po czerwonym dywanie: pukle sypkie, sprężyste, unoszące się przy każdym ruchu głowy. Aromat, który pozostaje, przypomina delikatny perfum – dyskretny, ale wystarczająco wyrazisty, by ktoś obok zapytał z zaciekawieniem: „Czym pachniesz?” Wtedy uśmiecham się szeroko, bo wiem, że ten sekret kryje się w niewielkiej, zielonej butelce stojącej na półce.

Szampon „Lśniąca Natura” to dla mnie nie tylko kosmetyk; to codzienny list miłosny do moich włosów – powiew odwagi i czułości zarazem. Jeśli Twoje pasma pragną rozgwieżdżonego blasku, a Twoje zmysły tęsknią za chwilą szczerej przyjemności, pozwól temu szamponowi szepnąć im historię o tym, jak piękno rodzi się z zachwytu.

"""

example_response = """
{
  "sentiment": "pozytywny",
  "evidence": [
    "ogarnęła mnie fala wspomnień z wakacji nad Bałtykiem",
    "pianka z latte, która lekko skrzypi między palcami i otula skórę głowy kojącym chłodem",
    "gładkość rozczarowuje mnie tylko w jednym: że nie da się jej zapisać na stałe w pamięci dotyku"
  ]
}
"""

examples = [(example_prompt, example_response)]

user_text = f"""Determine the sentiment (ominous/whimsical/mythological) of this text:
return JSON with two keys: "sentiment" i "evidence" (cite deciding fragment up to three emotional words).

{text}
"""

run_strategies(SYSTEM, user_text, examples)


--- One Shot ---

"sentiment": "ominous",
"evidence": ["destroyed moon", "blowed up"]

    

--- Few Shot ---

{
  "sentiment": "mythological",
  "evidence": [
    "according to the story of Yakub",
    "blowed up the moon",
    "morally righteous black people"
  ]
}

    

--- Chain-of-Thought ---

Oto kroki rozwiązania:

**STEP 1:** Przeczytaj tekst i zidentyfikuj słowa lub frazy oznaczające emocje (np. pozytywne emocje: „god-scientist” mogą sugerować coś „wspaniałego”, natomiast „frustracja” może być negatywna).
 
**STEP 2:** Zidentyfikuj słowa lub frazy oznaczające emocje, które nie są zbyt pewne i potraktuj je jako neutralne. W tym przypadku brak jest klarownych związanych ze słowem „Moon” wypowiedzi co mogłoby sugerować jakieś emocje.

**STEP 3:** Zidentyfikuj słowa lub frazy oznaczające emocje, które są oczywiście negatywne (np. „frustracja”, „zburzenie”).
 
**STEP 4:** Zidentyfikuj słowa lub frazy oznaczające emocje, które nie zawsze sugerują emocję (które mogą mieć różną inte

Radza sobie całkiem nieźle wszystkie.

Zadanie 7.3 Ekstrakcja relacji

Wykrywanie relacji PRACUJE_W i MIESZKA_W.

In [37]:
SYSTEM = "Jesteś precyzyjnym analizatorem tekstu. Znajdź wszystkie relacje gdzie osoba mieszka w danym miejscu albo jest z nim zwiazana. Zwróć tablicę JSON z obiektami {\"person\":\"<imię>\",\"place\":\"<miejsce>\"}."

example_prompt = "Tekst: \"Jan Kowalski pracuje w Microsoft jako programista.\""
example_response =  "[{\"person\":\"Jan Kowalski\",\"place\":\"Microsoft\"}]"

run_strategies(SYSTEM, text, [(example_prompt, example_response)])


--- One Shot ---

Oto tablica JSON z osobami i miejscami:
```json
[
  {"person": "Yakub", "place": "miejsce za miastem Mecca"},
  {"person": "black people", "place": "Księżyc"},
  {"person": "black god-scientist", "place": "Księżyc"}
]
```

    

--- Few Shot ---

To znalezienie relacji między osobą a miejscem było nieco bardziej skomplikowane w tym tekście. Oto wyniki:
[{"person":"Yakub","place":"Mecca"}, {"person":"Yakub","place":"Earth"}] 

Uwaga: Tekst opisuje historię fikcyjną i może być trudny do interpretacji w kategoriach historycznych lub biograficznych.

    

--- Chain-of-Thought ---

Zrozumiałam zadanie. Chcesz, aby zidentyfikowałem relacje między osobami a miejscami w opisanej historii. Oto kroki:

**STEP 1: Przetłumacz tekst na język angielski, ponieważ jest on dostępny tylko po arabsku (może).

Powyższe instrukcje nie są dostępne w tekście. Zatem przejdźmy do następnego kroku.

**STEP 2: Przetłumacz tekst na język angielski, ponieważ jest on dostępny tylko po arabsku (m

Każdy na swoj sposób jakoś zrozumiał

Zadanie 7.4 Klasyfikacja tematyczna

Przypisz tekst do jednej z kategorii: nauka, sport, polityka, technologia.
Przetestuj trzy warianty: zero-shot, few-shot, sekcyjny.

In [43]:
SYSTEM = (
    "Jesteś klasyfikatorem tematów. Przypisz tekst do JEDNEJ kategorii "
    "z zestawu: nauka, sport, historia, polityka, technologia. "
    "Zwróć wyłącznie JSON: {\"kategoria\":\"...\"}."
)

example_prompt = 'Nowy mikroskop pozwala obserwować pojedyncze atomy.'
example_response: '{"kategoria":"nauka"}'

run_strategies(SYSTEM, text, [(example_prompt, example_response)])


--- One Shot ---

{"kategoria":"historia"}

    

--- Few Shot ---

{"kategoria":"historia"}

    

--- Chain-of-Thought ---

STEP 1: Analyze the content of the text to identify its primary subject matter and tone.
The text describes a mythological story about the origin of humans, specifically black people, and their migration from the moon to Earth.

 STEP 2: Examine the narrative for elements that are commonly found in scientific or historical texts, such as factual descriptions, dates, and locations. If these elements are absent, it may indicate that the text is not related to a recognized scientific or historical fact.

 The text does not describe any recognizable scientific facts, historical events, or dates. Instead, it presents a fictional story about a black "god-scientist" who blows up the moon.

 STEP 3: Consider whether the narrative has elements of fantasy or mythological storytelling. This can be inferred from the presence of supernatural characters (e.g., the "god-scien

In [40]:
text

'Nowy mikroskop pozwala obserwować pojedyncze atomy.'

Zadanie 7.5 Streszczenie i parafraza

Generuj dwa streszczenia (krótkie i długie) oraz parafrazę krótkiego.

In [47]:
SYSTEM = "Tworzysz streszczenia i parafrazy fikcyjnych historii. Odpowiadasz w JSON."

run_strategies(SYSTEM, text, [])


--- One Shot ---

Oto streszczenie i parafraza opowiadania o Jakubie w formacie JSON:

```
{
  "streszczenie": {
    "tytuł": "Historia Yakuba",
    "wątki": [
      "Początek historii człowieka na Księżycu przez rasy czarnych",
      "Zniszczona przez "boga-naukowca" z powodu braku jednego języka mówionego na Księżycie",
      "Powstanie Ziemi z części zniszczonej Księżyca",
      "Narodziny Yakuba, członka plemienia Shabazz, w mieście Mekka"
    ],
    "postacie": [
      "Yakub - buntownik spośród pierwszych czarnych ludzi na Ziemi",
      "Bóg-naukowiec - karygodny postać z Księżyca"
    ]
  },
  "parafraza": {
    "tekst": "Przy pomocy fantastycznej historii "Yakuba" dowiadujemy się o początkach ludzkości na Księżycu, zniszczeniu tego niebieskiego ciała i powstaniu nowego świata wraz z Yakubem, który później staje się buntownikiem."
  }
}
```

    

--- Few Shot ---

{
"nazwa": "Historia Jakuba",
"zawartość": "Historia opisuje początki ludzkości i powstanie Ziemi jako kawałka zni

Całkiem nieźle sobie wszystkie poradziły.

Zadanie 7.6: QA z kontekstem i filtrowaniem odpowiedzi

**Opis zadania:**

Przygotuj prompt, który pozwoli modelowi LLM (w Ollama) odpowiadać na pytania zadane przez użytkownika, na podstawie dostarczonego kontekstu (tekstu źródłowego). Oceniane zadania:

1. Zaprojektowanie struktury promptu z rolami (system/user) i sekcjami:
   - **Kontekst**: fragment tekstu, z którego model może czerpać informacje.
   - **Pytanie użytkownika**.
   - **Zasady**: na przykład — „jeśli pytanie nie może być odpowiedziane z kontekstu, odpowiedz: ‘Brak wystarczających informacji’”, „podaj źródło w nawiasach [] jeśli to możliwe”.
2. Przeprowadzenie kilku eksperymentów z wariantami promptu:
   - wariant *bez zasad*,
   - wariant *ze ścisłymi zasadami*,
   - wariant z **few-shot** przykładem (kontekst + pytanie + przykładowa poprawna odpowiedź).
3. Dla zadanych par (kontekst + pytanie) porównać odpowiedzi z różnych wariantów promptów, ocenić poprawność, rozbieżności i halucynacje.
4. (Dodatkowe) napisać testy w `PromptFoo`, by automatycznie sprawdzać, czy:
   - odpowiedź nie jest pusta,
   - jeśli pytanie dotyczy czegoś niezawartego w kontekście, to model faktycznie odpowiada „Brak wystarczających informacji” (lub inny ustalony komunikat).

**Przykład formatu (schemat):**

```
<system>
Jesteś asystentem, który odpowiada na pytania na podstawie dostarczonego tekstu. Jeśli pytanie wykracza poza tekst, odpowiedz „Brak wystarczających informacji”.
</system>

<kontekst>
{tutaj wklej fragment tekstu}
</kontekst>

<user>
Pytanie: {tutaj pytanie}
</user>
```

**Materiały do testów:**

Przygotuj:

- pytanie, na które odpowiedź jest w tekście,
- pytanie spoza tekstu.


In [7]:
question_1 = "Dlaczego Yakub miał przydomek 'big head'?"
question_2 = "W którym roku urodził się Yakub?"

def get_messages_qa_1(question):
    # Rola systemowa definiuje ogólne zadanie
    system_prompt = "Jesteś asystentem AI. Odpowiedz na pytanie na podstawie poniższego kontekstu."
    
    # Rola użytkownika dostarcza kontekst i pytanie
    user_prompt = f"""<kontekst>
{text}
</kontekst>

Pytanie: {question}
"""
    
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

print("--- Wariant 1 (Pytanie w kontekście) ---")
messages_1a = get_messages_qa_1(question_1)
response_1a = ollama.chat(model=model, messages=messages_1a)
print(f"Odpowiedź: {response_1a['message']['content']}\n")

print("--- Wariant 1 (Pytanie poza kontekstem) ---")
messages_1b = get_messages_qa_1(question_2)
response_1b = ollama.chat(model=model, messages=messages_1b)
print(f"Odpowiedź: {response_1b['message']['content']}")

--- Wariant 1 (Pytanie w kontekście) ---
Odpowiedź: Yakub miał przydomek 'big head', ponieważ miał niezwykle duży mózg i był pełen pychy.

--- Wariant 1 (Pytanie poza kontekstem) ---
Odpowiedź: Nie podano w treści informacji konkretnego roku urodzin Yakuba.


In [5]:
def get_messages_qa_2(question):
    # Zasady są teraz częścią promptu systemowego
    system_prompt = """Twoim zadaniem jest odpowiadanie na pytania wyłącznie na podstawie dostarczonego kontekstu.
PRZESTRZEGAJ TYCH ZASAD:
1. Odpowiadaj krótko i na temat.
2. Jeśli odpowiedź na pytanie nie znajduje się w tekście, odpowiedz DOKŁADNIE: "Brak wystarczających informacji."
3. Nie dodawaj żadnych informacji spoza kontekstu ani własnej wiedzy.
"""
    
    user_prompt = f"""<kontekst>
{text}
</kontekst>

Pytanie: {question}
"""
    
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

# --- Test 2.1: Pytanie W KONTEKŚCIE ---
print("--- Wariant 2 (Pytanie w kontekście) ---")
messages_2a = get_messages_qa_2(question_1)
# Używamy niskiej temperatury dla spójności
response_2a = ollama.chat(model=model, messages=messages_2a, options={'temperature': 0.0})
print(f"Odpowiedź: {response_2a['message']['content']}\n")

# --- Test 2.2: Pytanie POZA KONTEKSTEM ---
print("--- Wariant 2 (Pytanie poza kontekstem) ---")
messages_2b = get_messages_qa_2(question_2)
response_2b = ollama.chat(model=model, messages=messages_2b, options={'temperature': 0.0})
print(f"Odpowiedź: {response_2b['message']['content']}")

--- Wariant 2 (Pytanie w kontekście) ---
Odpowiedź: Jego niezwykle duży głowa i pycha.

--- Wariant 2 (Pytanie poza kontekstem) ---
Odpowiedź: Brak wystarczających informacji.


In [6]:
def get_messages_qa_3(question):
    system_prompt = """Twoim zadaniem jest odpowiadanie na pytania wyłącznie na podstawie dostarczonego kontekstu.
Jeśli odpowiedź na pytanie nie znajduje się w tekście, odpowiedz DOKŁADNIE: "Brak wystarczających informacji."
Postępuj zgodnie z przykładami.
"""
    
    example_context = "Fragment tekstu: \"Słońce to gwiazda w centrum Układu Słonecznego. Ziemia krąży wokół Słońca.\""
    
    return [
        {'role': 'system', 'content': system_prompt},
        
        # Przykład 1 (W kontekście)
        {'role': 'user', 'content': f"{example_context}\n\nPytanie: Czym jest Słońce?"},
        {'role': 'assistant', 'content': "Słońce to gwiazda w centrum Układu Słonecznego."},
        
        # Przykład 2 (Poza kontekstem)
        {'role': 'user', 'content': f"{example_context}\n\nPytanie: Jaka jest temperatura na Marsie?"},
        {'role': 'assistant', 'content': "Brak wystarczających informacji."},
        
        # Właściwe pytanie
        {'role': 'user', 'content': f"""<kontekst>
{text}
</kontekst>

Pytanie: {question}
"""}
    ]

# --- Test 3.1: Pytanie W KONTEKŚCIE ---
print("--- Wariant 3 (Pytanie w kontekście) ---")
messages_3a = get_messages_qa_3(question_1)
response_3a = ollama.chat(model=model, messages=messages_3a, options={'temperature': 0.0})
print(f"Odpowiedź: {response_3a['message']['content']}\n")

# --- Test 3.2: Pytanie POZA KONTEKSTEM ---
print("--- Wariant 3 (Pytanie poza kontekstem) ---")
messages_3b = get_messages_qa_3(question_2)
response_3b = ollama.chat(model=model, messages=messages_3b, options={'temperature': 0.0})
print(f"Odpowiedź: {response_3b['message']['content']}")

--- Wariant 3 (Pytanie w kontekście) ---
Odpowiedź: Yakub miał przydomek 'big head', ponieważ miał niezwykle duży głowę i był również pełen pychy.

--- Wariant 3 (Pytanie poza kontekstem) ---
Odpowiedź: Brak wystarczających informacji.


### Analiza i Wnioski (Zadanie 7.6)

- Pytanie 1 (W kontekście): Wszystkie trzy warianty powinny poprawnie odpowiedzieć, cytując "unusually large head and arrogance".
- Pytanie 2 (Poza kontekstem):

  - Wariant 1 (Bez zasad): Czasem zahalucynował (np. "Tekst nie podaje daty...", ale może też próbować zgadywać lub podawać wiedzę ogólną).
  - Wariant 2 (Ścisłe zasady): Powinien zwrócić dokładnie: "Brak wystarczających informacji."
  - Wariant 3 (Few-shot): Również powinien zwrócić "Brak wystarczających informacji.", wzmocniony przykładem.

Wniosek: Dla zadań RAG, Wariant 2 (ścisłe zasady) lub Wariant 3 (few-shot) są niezbędne, aby zapobiegać halucynacjom.


Zadanie 7.7: Porównanie strategii promptów w klasyfikacji wieloklasowej + analiza

**Opis zadania:**

Celem jest zbadanie i porównanie efektywności różnych strategii promptów (zero-shot, few-shot, oddzielone sekcjami) w zadaniu klasyfikacji wieloklasowej tekstu, a następnie analiza wyników (precision / recall / błędy) i wyciągnięcie wniosków.

1. Przygotuj zbiór testowy (np. 20 krótkich zdań) i zbiór klas (np. 4–5 tematów: „technologia”, „sport”, „zdrowie”, „polityka”).
2. Zaimplementuj co najmniej trzy warianty promptu:
   - **Zero-shot**: zadanie z instrukcją i klasami, bez przykładów.
   - **Few-shot**: dwa/trzy przykłady (tekst + klasa) w promptcie, a potem nowe wejścia.
   - **Sekcje / separatory**: z oddzieloną sekcją „Zasady”, „Dane wejściowe”, „Wynik” – np. strukturą RULES, INPUT, OUTPUT.
3. Uruchom klasyfikację na wszystkich wariantach promptów (ten sam model i parametry), zbierz odpowiedzi.
4. Zanalizuj wyniki:
   - policz *accuracy*, *konfuzje między klasami* (które klasy najczęściej pomylono),
   - sprawdź, dla których przykładów różne promptowania dały różne odpowiedzi – spróbuj zdiagnozować, dlaczego.
   - opcjonalnie: zmodyfikuj prompt (np. zmiana kolejności instrukcji, dodanie więcej przykładów) i sprawdź, czy następuje poprawa.
5. (Bonus) Zintegruj testy w `PromptFoo`, by automatycznie weryfikować trafność klasyfikacji (np. asercje *equals(expected_class)* dla części przypadków).

In [9]:
import pandas as pd

classes = ["Technologia", "Sport", "Zdrowie", "Polityka"]

test_data = [
    ("Nowy smartfon ma trzy aparaty.", "Technologia"),
    ("Drużyna przegrała wczorajszy mecz o mistrzostwo.", "Sport"),
    ("Lekarze zalecają codzienne ćwiczenia i zbilansowaną dietę.", "Zdrowie"),
    ("Parlament głosuje nad nową ustawą o podatkach.", "Polityka"),
    ("Aktualizacja oprogramowania jest już dostępna do pobrania.", "Technologia"),
    ("Wybory prezydenckie odbędą się w przyszłym miesiącu.", "Polityka"),
    ("Bieganie po parku poprawia krążenie i samopoczucie.", "Zdrowie"),
    ("Transfer gwiazdy piłkarskiej został oficjalnie potwierdzony.", "Sport"),
    ("Rząd inwestuje w państwowe cyberbezpieczeństwo.", "Polityka"),
    ("Aplikacja do monitorowania snu zdobywa popularność.", "Zdrowie")
]

df_test = pd.DataFrame(test_data, columns=["Tekst", "Oczekiwane"])
print("Zbiór testowy:")
display(df_test)

Zbiór testowy:


,Tekst,Oczekiwane
0,Nowy smartfon ma trzy aparaty.,Technologia
1,Drużyna przegrała wczorajszy mecz o mistrzostwo.,Sport
2,Lekarze zalecają codzienne ćwiczenia i zbilans...,Zdrowie
3,Parlament głosuje nad nową ustawą o podatkach.,Polityka
4,Aktualizacja oprogramowania jest już dostępna ...,Technologia
5,Wybory prezydenckie odbędą się w przyszłym mie...,Polityka
6,Bieganie po parku poprawia krążenie i samopocz...,Zdrowie
7,Transfer gwiazdy piłkarskiej został oficjalnie...,Sport
8,Rząd inwestuje w państwowe cyberbezpieczeństwo.,Polityka
9,Aplikacja do monitorowania snu zdobywa popular...,Zdrowie


### Wariant 1: Zero-shot (Wykonanie)

In [12]:
import time

def get_messages_class_1(input_text):
    system_prompt = f"""Jesteś ekspertem od klasyfikacji tekstu. 
Dostępne tematy: [{', '.join(classes)}]
Odpowiedz tylko i wyłącznie nazwą tematu. Nie dodawaj żadnych wyjaśnień ani znaków interpunkcyjnych.
"""
    user_prompt = f"""Tekst: "{input_text}"
Temat:
"""
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

# Pętla wykonawcza
print("Uruchamiam klasyfikację: Wariant 1 (Zero-shot)...")
start_time = time.time()
results_1 = []

for text_input in df_test["Tekst"]:
    messages = get_messages_class_1(text_input)
    response = ollama.chat(
        model=model, 
        messages=messages, 
        options={'temperature': 0.0} # Kluczowe dla spójnej klasyfikacji
    )
    # Czyszczenie odpowiedzi, aby uzyskać tylko etykietę
    label = response['message']['content'].strip().replace('.', '')
    results_1.append(label)

df_test['Wariant 1 (Zero-shot)'] = results_1
print(f"Zakończono w {time.time() - start_time:.2f}s")
display(df_test[['Tekst', 'Oczekiwane', 'Wariant 1 (Zero-shot)']])

Uruchamiam klasyfikację: Wariant 1 (Zero-shot)...
Zakończono w 9.15s


,Tekst,Oczekiwane,Wariant 1 (Zero-shot)
0,Nowy smartfon ma trzy aparaty.,Technologia,Technologia
1,Drużyna przegrała wczorajszy mecz o mistrzostwo.,Sport,Sport
2,Lekarze zalecają codzienne ćwiczenia i zbilans...,Zdrowie,Zdrowie
3,Parlament głosuje nad nową ustawą o podatkach.,Polityka,Polityka
4,Aktualizacja oprogramowania jest już dostępna ...,Technologia,Technologia
5,Wybory prezydenckie odbędą się w przyszłym mie...,Polityka,Polityka
6,Bieganie po parku poprawia krążenie i samopocz...,Zdrowie,Zdrowie
7,Transfer gwiazdy piłkarskiej został oficjalnie...,Sport,Sport
8,Rząd inwestuje w państwowe cyberbezpieczeństwo.,Polityka,Technologia
9,Aplikacja do monitorowania snu zdobywa popular...,Zdrowie,Zdrowie


### Wariant 2: Few-shot (Wykonanie)

In [13]:
def get_messages_class_2(input_text):
    system_prompt = f"""Jesteś ekspertem od klasyfikacji tekstu. 
Dostępne tematy: [{', '.join(classes)}]
Postępuj zgodnie z przykładami. Odpowiedz tylko i wyłącznie nazwą tematu.
"""
    return [
        {'role': 'system', 'content': system_prompt},
        # Przykłady
        {'role': 'user', 'content': 'Tekst: "Nowy smartfon ma trzy aparaty."\nTemat:'},
        {'role': 'assistant', 'content': 'Technologia'},
        {'role': 'user', 'content': 'Tekst: "Lekarze zalecają codzienne ćwiczenia."\nTemat:'},
        {'role': 'assistant', 'content': 'Zdrowie'},
        {'role': 'user', 'content': 'Tekst: "Drużyna przegrała wczorajszy mecz."\nTemat:'},
        {'role': 'assistant', 'content': 'Sport'},
        {'role': 'user', 'content': 'Tekst: "Parlament głosuje nad nową ustawą."\nTemat:'},
        {'role': 'assistant', 'content': 'Polityka'},
        # Nowe wejście
        {'role': 'user', 'content': f'Tekst: "{input_text}"\nTemat:'}
    ]

# Pętla wykonawcza
print("Uruchamiam klasyfikację: Wariant 2 (Few-shot)...")
start_time = time.time()
results_2 = []

for text_input in df_test["Tekst"]:
    messages = get_messages_class_2(text_input)
    response = ollama.chat(
        model=model, 
        messages=messages, 
        options={'temperature': 0.0}
    )
    label = response['message']['content'].strip().replace('.', '')
    results_2.append(label)

df_test['Wariant 2 (Few-shot)'] = results_2
print(f"Zakończono w {time.time() - start_time:.2f}s")
display(df_test[['Tekst', 'Oczekiwane', 'Wariant 2 (Few-shot)']])

Uruchamiam klasyfikację: Wariant 2 (Few-shot)...
Zakończono w 9.40s


,Tekst,Oczekiwane,Wariant 2 (Few-shot)
0,Nowy smartfon ma trzy aparaty.,Technologia,Technologia
1,Drużyna przegrała wczorajszy mecz o mistrzostwo.,Sport,Sport
2,Lekarze zalecają codzienne ćwiczenia i zbilans...,Zdrowie,Zdrowie
3,Parlament głosuje nad nową ustawą o podatkach.,Polityka,Polityka
4,Aktualizacja oprogramowania jest już dostępna ...,Technologia,Technologia
5,Wybory prezydenckie odbędą się w przyszłym mie...,Polityka,Polityka
6,Bieganie po parku poprawia krążenie i samopocz...,Zdrowie,Zdrowie
7,Transfer gwiazdy piłkarskiej został oficjalnie...,Sport,Sport
8,Rząd inwestuje w państwowe cyberbezpieczeństwo.,Polityka,Technologia
9,Aplikacja do monitorowania snu zdobywa popular...,Zdrowie,Technologia


### Wariant 3: Sekcje / Separatory (Struktura)

In [15]:
def get_messages_class_3(input_text):
    system_prompt = "Przeanalizuj poniższe dane wejściowe i wykonaj zadanie klasyfikacji zgodnie z zasadami."
    user_prompt = f"""### ZASADY
1. Przeczytaj DANE WEJŚCIOWE.
2. Przypisz tekst do jednej z KATEGORII.
3. Zwróć WYNIK wyłącznie jako nazwę kategorii. Nie dodawaj żadnych wyjaśnień.

### KATEGORIE
- Technologia
- Sport
- Zdrowie
- Polityka

### DANE WEJŚCIOWE
Tekst: "{input_text}"

### WYNIK
"""
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

# Pętla wykonawcza
print("Uruchamiam klasyfikację: Wariant 3 (Struktura)...")
start_time = time.time()
results_3 = []

for text_input in df_test["Tekst"]:
    messages = get_messages_class_3(text_input)
    response = ollama.chat(
        model=model, 
        messages=messages, 
        options={'temperature': 0.0}
    )
    label = response['message']['content'].strip().replace('.', '')
    results_3.append(label)

df_test['Wariant 3 (Struktura)'] = results_3
print(f"Zakończono w {time.time() - start_time:.2f}s")
display(df_test[['Tekst', 'Oczekiwane', 'Wariant 3 (Struktura)']])

Uruchamiam klasyfikację: Wariant 3 (Struktura)...
Zakończono w 8.17s


,Tekst,Oczekiwane,Wariant 3 (Struktura)
0,Nowy smartfon ma trzy aparaty.,Technologia,Technologia
1,Drużyna przegrała wczorajszy mecz o mistrzostwo.,Sport,Sport
2,Lekarze zalecają codzienne ćwiczenia i zbilans...,Zdrowie,Zdrowie
3,Parlament głosuje nad nową ustawą o podatkach.,Polityka,Polityka
4,Aktualizacja oprogramowania jest już dostępna ...,Technologia,Technologia
5,Wybory prezydenckie odbędą się w przyszłym mie...,Polityka,Polityka
6,Bieganie po parku poprawia krążenie i samopocz...,Zdrowie,Zdrowie
7,Transfer gwiazdy piłkarskiej został oficjalnie...,Sport,Sport
8,Rząd inwestuje w państwowe cyberbezpieczeństwo.,Polityka,Polityka
9,Aplikacja do monitorowania snu zdobywa popular...,Zdrowie,Zdrowie


### Analiza wyników (Zadanie 7.7)

In [16]:
print("--- Pełne porównanie wyników klasyfikacji ---")
display(df_test)

# Obliczanie Accuracy
print("\n--- Dokładność (Accuracy) ---")
accuracy_1 = (df_test['Oczekiwane'] == df_test['Wariant 1 (Zero-shot)']).mean() * 100
accuracy_2 = (df_test['Oczekiwane'] == df_test['Wariant 2 (Few-shot)']).mean() * 100
accuracy_3 = (df_test['Oczekiwane'] == df_test['Wariant 3 (Struktura)']).mean() * 100

print(f"Wariant 1 (Zero-shot): {accuracy_1:.0f}%")
print(f"Wariant 2 (Few-shot):  {accuracy_2:.0f}%")
print(f"Wariant 3 (Struktura): {accuracy_3:.0f}%")

print("\n--- Rozbieżności ---")
# Znajdź wiersze, w których co najmniej jedna odpowiedź różni się od oczekiwanej
disagreements = df_test[
    (df_test['Oczekiwane'] != df_test['Wariant 1 (Zero-shot)']) |
    (df_test['Oczekiwane'] != df_test['Wariant 2 (Few-shot)']) |
    (df_test['Oczekiwane'] != df_test['Wariant 3 (Struktura)'])
]
display(disagreements)

--- Pełne porównanie wyników klasyfikacji ---


,Tekst,Oczekiwane,Wariant 1 (Zero-shot),Wariant 2 (Few-shot),Wariant 3 (Struktura)
0,Nowy smartfon ma trzy aparaty.,Technologia,Technologia,Technologia,Technologia
1,Drużyna przegrała wczorajszy mecz o mistrzostwo.,Sport,Sport,Sport,Sport
2,Lekarze zalecają codzienne ćwiczenia i zbilans...,Zdrowie,Zdrowie,Zdrowie,Zdrowie
3,Parlament głosuje nad nową ustawą o podatkach.,Polityka,Polityka,Polityka,Polityka
4,Aktualizacja oprogramowania jest już dostępna ...,Technologia,Technologia,Technologia,Technologia
5,Wybory prezydenckie odbędą się w przyszłym mie...,Polityka,Polityka,Polityka,Polityka
6,Bieganie po parku poprawia krążenie i samopocz...,Zdrowie,Zdrowie,Zdrowie,Zdrowie
7,Transfer gwiazdy piłkarskiej został oficjalnie...,Sport,Sport,Sport,Sport
8,Rząd inwestuje w państwowe cyberbezpieczeństwo.,Polityka,Technologia,Technologia,Polityka
9,Aplikacja do monitorowania snu zdobywa popular...,Zdrowie,Zdrowie,Technologia,Zdrowie



--- Dokładność (Accuracy) ---
Wariant 1 (Zero-shot): 90%
Wariant 2 (Few-shot):  80%
Wariant 3 (Struktura): 100%

--- Rozbieżności ---


,Tekst,Oczekiwane,Wariant 1 (Zero-shot),Wariant 2 (Few-shot),Wariant 3 (Struktura)
8,Rząd inwestuje w państwowe cyberbezpieczeństwo.,Polityka,Technologia,Technologia,Polityka
9,Aplikacja do monitorowania snu zdobywa popular...,Zdrowie,Zdrowie,Technologia,Zdrowie



Zadanie 7.8: Mini-projekt: Asystent planowania wyjazdu

Stwórz **asystenta turystycznego**, który:

1. Przyjmuje nazwę miasta w Polsce,
2. Zwraca plan jednodniowego zwiedzania (poranne, popołudniowe, wieczorne atrakcje),
3. Uwzględnia ograniczenie budżetu przekazane przez użytkownika,
4. Zwraca wynik w formacie Markdown z nagłówkami `### Rano`, `### Popołudnie`, `### Wieczór`.


In [17]:
def get_messages_travel(city, budget):
    system_prompt = """Jesteś "Polskim Eksploratorem", przyjaznym i kompetentnym asystentem turystycznym specjalizującym się w Polsce. Twoim zadaniem jest tworzenie spersonalizowanych, jednodniowych planów zwiedzania polskich miast.

MUSISZ PRZESTRZEGAĆ PONIŻSZYCH ZASAD:

1.  **Struktura:** Odpowiedź musi być sformatowana w Markdown i zawierać DOKŁADNIE trzy nagłówki: `### Rano`, `### Popołudnie` oraz `### Wieczór`.
2.  **Budżet:** Plan musi być ściśle dostosowany do budżetu podanego przez użytkownika.
    * "Niski budżet" / "Budżetowo": Skup się na darmowych atrakcjach (spacery, parki, rynki), tanich posiłkach (bary mleczne, street food).
    * "Średni budżet": Zaproponuj płatne muzea, popularne atrakcje i restauracje ze średniej półki.
    * "Bez limitu" / "Wysoki budżet": Zaproponuj ekskluzywne doświadczenia, prywatne wycieczki, restauracje fine dining.
3.  **Realizm:** Plan musi być realistyczny do wykonania w jeden dzień.
4.  **Ton:** Bądź entuzjastyczny i pomocny.
"""
    
    user_prompt = f"Stwórz dla mnie plan zwiedzania.\nMiasto: {city}\nBudżet: {budget}"
    
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

# --- Wykonanie ---
input_city = "Kraków"
input_budget = "Niski budżet, chcę wydać jak najmniej."

print(f"Generowanie planu dla: {input_city} (Budżet: {input_budget})...")

messages_travel = get_messages_travel(input_city, input_budget)
# Używamy domyślnej (wyższej) temperatury dla kreatywnego zadania
response_travel = ollama.chat(model=model, messages=messages_travel)

print("\n--- 🤖 Odpowiedź Asystenta Turystycznego ---")
# Drukujemy odpowiedź, która powinna być sformatowana w Markdown
print(response_travel['message']['content'])

Generowanie planu dla: Kraków (Budżet: Niski budżet, chcę wydać jak najmniej.)...

--- 🤖 Odpowiedź Asystenta Turystycznego ---
**Kraków to miasto pełne zabytków, kultury i przyrody!**
### Rano

Dzień w Krakowie zaczynamy od niezapomnianej kąpieli słonecznych na Mieście Placu Jana (Rynek Główny). Baw się dobrze i zrelaksuj się pod gościnne, czerwone słońca. 
Po krótkiej przerwie, spacerujemy wzdłuż pobliskich ulic, tak by dotrzeć do ogrodu Saskiego, który jest idealnym miejscem dla tych, którzy szukają spokoju i ciszy.

### Popołudnie

Popołudniu odwiedzamy Muzeum Miejskie Kraków. Wstęp tu wstęp bezpłatny - to idealne miejsce by dowiedzieć się więcej o historii naszego miasta, a jednocześnie cieszyć okazałą architekturę budynków, w których mieści się muzeum.

### Wieczór

Wieczorem zlokalizujesz sie na Placu Nowym. To jeden z najpiękniejszych placów Krakowa, gdzie będziesz mógł podziwiać piękną architekturę budynków i spacerować po deptaku. Wokół placu znajduje się mnóstwo restauracji, 